#Analisador de Processo de Compra

Objetivo 1: Construir as mensagens dos emails para envio das notas de empenho

Objetivo 2: Listar os produtos fracassados e checar se todos os produtos homologados pela DG constam nas notas de empenho anexadas ao processo

Objetivo 3: Criar uma Planilha para acompanhamento das entregas dos produtos pelos fornecedores

Objetivo 4: Checar se todos os pagamentos aos fornecedores já foram feitos para poder arquivar o processo

In [ ]:
!pip install nltk
!pip install PyPDF2



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Carrega o processo selecionado para "textPDF"

In [ ]:
import PyPDF2

medicamentos = 'SEI_03789_2024.pdf'
mat_hosp_odon = 'SEI_03699_2024.pdf'
outros1 = 'SEI_03930_2024.pdf'

with open(f'/content/drive/MyDrive/Colab Notebooks/Processo_compras/{medicamentos}', 'rb') as file:
    reader = PyPDF2.PdfReader(file)

    textPDF = ''
    for page in reader.pages:
        textPDF += page.extract_text()

    num_pages = len(reader.pages)

print(f"O documento contém {num_pages} páginas.")

O documento contém 458 páginas.


In [ ]:
#capturar os dados principais das notas de empenho

import nltk
nltk.download('punkt')
# Dividir o texto em linhas
lines = nltk.tokenize.sent_tokenize(textPDF)

notas_de_empenho = [line for line in lines if "Ano Tipo Número70008 TRIBUNAL REGIONAL ELEITORAL DO RIO G.DO NORTE REAL" in line]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Capturar nas Notas de Empenho o CNPJ e o nome da empresa, o número da Nota de Empenho e o valor.
São criados tantos dicionários quanto for o número de notas de empenho no processo e os dicionários são adicionados a uma lista

In [ ]:
import pprint

lista_dict = []

for nota_emp in notas_de_empenho:

  # Criar o dicionário
  data_dict = {}

  # Extrair as informações
  lines = nota_emp.splitlines()

  # Encontrar e adicionar 'NE'
  for line in lines:
      if "NE" in line:
          parts = line.split()
          data_dict['NE'] = parts[parts.index('NE') + 1]

  # Encontrar e adicionar 'Empresa' e 'CNPJ'
  for line in lines:
      if "/00" in line:
          parts = line.split()
          data_dict['CNPJ'] = parts[0]
          data_dict['Empresa'] = ' '.join(parts[1:])
          break  # Para evitar múltiplas capturas

  # Encontrar e adicionar 'Valor' usando referências
  for line in lines:
      if "0,0000" in line:
          # Capturar o valor que vem depois de "0,0000 "
          valor = line.split("0,0000 ")[1].split("\n")[0].strip()
          data_dict['Valor'] = valor
          break  # Para evitar múltiplas capturas

  # Adicionar o dicionário à lista
  lista_dict.append(data_dict)
# Exibir o dicionário
pprint.pprint(lista_dict)

[{'CNPJ': '05.792.645/0001-28CEP', 'Empresa': ''},
 {'CNPJ': '13.537.459/0001-70',
  'Empresa': '2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA',
  'NE': '322',
  'Valor': '4.569,02'}]


In [ ]:
def remover_dicionarios_por_chave(lista_dicionarios, chave_desejada):
  """Remove da lista de dicionários todos os dicionários que não começam com NE (não são Notas de Empenho)
  """
  lista_filtrada = []
  for dicionario in lista_dicionarios:
    if chave_desejada in dicionario:
      lista_filtrada.append(dicionario)
  return lista_filtrada

lista_de_dicionarios = remover_dicionarios_por_chave(lista_dict, "NE")
lista_de_dicionarios

[{'NE': '322',
  'CNPJ': '13.537.459/0001-70',
  'Empresa': '2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA',
  'Valor': '4.569,02'}]

Cria um DataFrame a partir do dicionário da lista de dicionários

In [ ]:
#criação do DataFrame df_Notas_de_empenho com os dados do dicionario lista_dict
import pandas as pd
df_Notas_de_empenho = pd.DataFrame(lista_de_dicionarios)
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor
0,322,13.537.459/0001-70,2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA,"4.569,02"


Capturar os emails e inserir no DataFrame

In [ ]:
#pegar os emails

import re

emails = []

for i in range (df_Notas_de_empenho.shape[0]):
  emails.append("nao_encontrado")

for i in range (df_Notas_de_empenho.shape[0]):
  # String a ser encontrada
  company_name = df_Notas_de_empenho['Empresa'][i]
  stop_phrase = 'Solicito análise das propostas'
 # stop_phrase = 'Validade da proposta'

  # Regex para encontrar um email
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
  # Encontrar a posição da frase que limita a busca
  stop_index = textPDF.find(stop_phrase)

  if stop_index != -1:
      # Obter o texto antes da frase de parada
      text_before_stop = textPDF[:stop_index]
      # Encontrar o nome da empresa no texto
      company_match_name = re.search(re.escape(company_name), text_before_stop)

      if company_match_name:
          # Obter o texto após a empresa
          text_after_company_name = text_before_stop[company_match_name.end():]
          # Procurar o primeiro email no texto subsequente
          email_match = re.search(email_pattern, text_after_company_name)
          if email_match:
              emails[i] = email_match.group()

  else:
      print("Frase de parada não encontrada no texto.")

for i in range (df_Notas_de_empenho.shape[0]):
  # String a ser encontrada
  company_cnpj = df_Notas_de_empenho['CNPJ'][i]
  stop_phrase = 'Solicito análise das propostas'

  # Regex para encontrar um email
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
  # Encontrar a posição da frase que limita a busca
  stop_index = textPDF.find(stop_phrase)

  if stop_index != -1:
      # Obter o texto antes da frase de parada
      text_before_stop = textPDF[:stop_index]

      # Encontrar o cnpj da empresa no texto
      company_match_cnpj = re.search(re.escape(company_cnpj), text_before_stop)

      if company_match_cnpj:
          # Obter o texto após a empresa
          text_after_company_cnpj = text_before_stop[company_match_cnpj.end():]
          # Procurar o primeiro email no texto subsequente
          email_match = re.search(email_pattern, text_after_company_cnpj)

          if email_match:
            if emails[i] == "nao_encontrado":
              emails[i] = email_match.group()


  else:
      print("Frase de parada não encontrada no texto.")

df_Notas_de_empenho['Email'] = [email.lower() for email in emails]
df_Notas_de_empenho


Frase de parada não encontrada no texto.
Frase de parada não encontrada no texto.


,NE,CNPJ,Empresa,Valor,Email
0,322,13.537.459/0001-70,2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA,"4.569,02",nao_encontrado


Fazer outra busca e listar todos os emails existentes no processo exceto os que contiverem "tre-rn"

In [ ]:
import re

# Expressão regular para encontrar emails
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Encontrar todos os emails no texto
emails = re.findall(email_pattern, textPDF)
emails = list(set(emails))

# Remover os emails que forem do 'tre-rn'
filtered_emails = [item for item in emails if 'tre-rn' not in item]

email_minusculas = []
for string in filtered_emails:
    email_minusculas.append(string.lower())

email_minusculas = list(set(email_minusculas))
# Exibir os emails encontrados
pprint.pprint(email_minusculas)


['sac@drogalider.com.br', 'sac@pmenos.com.br', 'comercial@bigmedica.com.br']


In [ ]:
def mostra_emails(lista_emails):
  for i in range (len(lista_emails)):
    print(f"{i+1}: {lista_emails[i]}")

In [ ]:
emails_verificados = []
for i in range (df_Notas_de_empenho.shape[0]):
  if df_Notas_de_empenho['Empresa'][i] not in emails_verificados:
    print(f"O email encontrado para a empresa {df_Notas_de_empenho['Empresa'][i]} é o email {df_Notas_de_empenho['Email'][i]}")
    necessidade_correcao = input("É necessário corrigir? (s/n)")
    necessidade_correcao = necessidade_correcao.lower()
    if necessidade_correcao == 's':
      mostra_emails(email_minusculas)
      n = int(input(f"Digite o número o email para setar na Empresa: {df_Notas_de_empenho['Empresa'][i]}\nCaso não esteja listado o email desejado digite 0\n"))
      #print("Caso não esteja listado o email desejado digite 0")

      emails_verificados.append(df_Notas_de_empenho['Empresa'][i])
      if n > 0:
          df_Notas_de_empenho.loc[df_Notas_de_empenho['Empresa'] == df_Notas_de_empenho['Empresa'][i], 'Email'] = email_minusculas[n-1]



df_Notas_de_empenho

O email encontrado para a empresa 2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA é o email nao_encontrado
É necessário corrigir? (s/n)s
1: sac@drogalider.com.br
2: sac@pmenos.com.br
3: comercial@bigmedica.com.br
Digite o número o email para setar na Empresa: 2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA
Caso não esteja listado o email desejado digite 0
3


,NE,CNPJ,Empresa,Valor,Email
0,322,13.537.459/0001-70,2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA,"4.569,02",comercial@bigmedica.com.br


Inserir manualmente **digitando** algum email que não tenha sido capturado

In [ ]:
#para inserir manualmente digitando algum email que não tenha sido capturado

empresas_nao_encontrado = df_Notas_de_empenho.loc[df_Notas_de_empenho['Email'] == 'nao_encontrado', 'Empresa']
if len(empresas_nao_encontrado) > 0:
  empresas_nao_encontrado.reset_index(drop=True, inplace=True)
  print("Os email das seguinte empresas não foram inseridos:")
  print(empresas_nao_encontrado)
  for i in range (len(empresas_nao_encontrado)):
    email_para_inserir = input(f"Digite o email da empresa {empresas_nao_encontrado[i]} para inserção manual: ")
    df_Notas_de_empenho.loc[df_Notas_de_empenho['Empresa'] == empresas_nao_encontrado[i], 'Email'] = email_para_inserir

df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor,Email
0,322,13.537.459/0001-70,2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA,"4.569,02",comercial@bigmedica.com.br


Início de criação das mensagens para envio das Notas de Empenho

In [ ]:

empresas = df_Notas_de_empenho['Empresa'].unique()  # Obter valores únicos da coluna "Empresa"

# Criar um dicionário com listas vazias, onde as chaves são os nomes das empresas
listas_empresas = {empresa: [] for empresa in empresas}

# Iterar sobre o DataFrame e preencher as listas com os números da coluna "NE"
for _, row in df_Notas_de_empenho.iterrows():
    empresa = row['Empresa']
    ne = row['NE']
    listas_empresas[empresa].append(ne)

# Exibir o dicionário preenchido
print(listas_empresas)



{'2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA': ['322']}


In [ ]:
from datetime import datetime

# Obter a data atual
current_date = datetime.now()

# Formatar o dia, mês e ano como strings
formatted_date = f"SAMS - TRE-RN, {current_date.day}/{current_date.month}/{current_date.year}."

# Função para criar e-mails
def criar_emails(empresas):
    emails = []
    for empresa, numeros in empresas.items():
        for numero in numeros:

            num_NE = f'{current_date.year}NE000{numero}'
            email = f"""Prezados(as) Senhores(as),

Encaminhamos a Nota de Empenho número {num_NE} (em anexo), à empresa {empresa}.
Informamos que o prazo para entrega do material é de até 30 (trinta) dias corridos, contados a partir da data de envio
desta Nota de Empenho.
Solicitamos especial atenção quanto ao PRAZO, LOCAL e HORÁRIO para entrega do material.
A entrega deverá ser realizada na Seção de Assistência Médica e Saúde Ocupacional (SAMS), localizada na Sede
Administrativa do TRE/RN. Endereço: Av. Rui Barbosa, 215. Tirol, Natal-RN. CEP: 59.015-290. Telefone: (84) 3654-
5399, nos horários de segunda a quinta-feira, das 13 às 18 horas ou na sexta-feira, das 8 às 13 horas.
FAVOR CONFIRMAR RECEBIMENTO DESTE E-MAIL enviando resposta para o email sams@tre-rn.jus.br

Atenciosamente,
Flávio Roberto Guerra Seabra
{formatted_date}
                      """
            emails.append(email)
    return emails

# Chamar a função e obter os e-mails
emails_gerados = criar_emails(listas_empresas)

# Nome do arquivo de saída
output_file = 'textos_emails.txt'

# Abrir o arquivo para escrita
with open(output_file, 'w', encoding='utf-8') as file:

# Exibir os e-mails gerados
    for email in emails_gerados:
        print(email)
        file.write(email)
        print("-" * 40)
        file.write("-" * 40)
        file.write("\n")

Prezados(as) Senhores(as),

Encaminhamos a Nota de Empenho número 2024NE000322 (em anexo), à empresa 2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA.
Informamos que o prazo para entrega do material é de até 30 (trinta) dias corridos, contados a partir da data de envio
desta Nota de Empenho.
Solicitamos especial atenção quanto ao PRAZO, LOCAL e HORÁRIO para entrega do material.
A entrega deverá ser realizada na Seção de Assistência Médica e Saúde Ocupacional (SAMS), localizada na Sede
Administrativa do TRE/RN. Endereço: Av. Rui Barbosa, 215. Tirol, Natal-RN. CEP: 59.015-290. Telefone: (84) 3654-
5399, nos horários de segunda a quinta-feira, das 13 às 18 horas ou na sexta-feira, das 8 às 13 horas.
FAVOR CONFIRMAR RECEBIMENTO DESTE E-MAIL enviando resposta para o email sams@tre-rn.jus.br

Atenciosamente,
Flávio Roberto Guerra Seabra
SAMS - TRE-RN, 24/10/2024.
                      
----------------------------------------


Início de captura dos números dos itens que restaram Desertos e Fracassados no processo licitatório

In [ ]:
import re

# Definindo o padrão para capturar o trecho desejado
pattern = r'(requisitos legais foram devidamente observados.*?êxito e aguardam adjudicação)'

# Encontrar o trecho entre os padrões
match = re.search(pattern, textPDF, re.DOTALL)

if match:
    # Obter o texto completo entre os padrões
    parecer_ajdg_homolog = match.group(1).strip()  # O texto entre os dois padrões
else:
    print("Trecho não encontrado.")

In [ ]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

tokens = word_tokenize(parecer_ajdg_homolog)

# Remover stopwords
stop_words = set(stopwords.words('portuguese'))
parecer_ajdg_homolog_tokenizado = [word for word in tokens if word.lower() not in stop_words]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
parecer_ajdg_homolog_tokenizado = [s for s in parecer_ajdg_homolog_tokenizado if s != ',']


In [ ]:
lista_desertos = []
lista_fracassados = []
capturar = False
cont = 0
# Iterar sobre a lista e substituir 'item' por 'itens'
for i in reversed(range(len(parecer_ajdg_homolog_tokenizado))):
    if parecer_ajdg_homolog_tokenizado[i].startswith('fracass') and cont == 0:
      capturar = True
      cont = 1
    if capturar == True:
      if isinstance(parecer_ajdg_homolog_tokenizado[i], str) and parecer_ajdg_homolog_tokenizado[i].isdigit():
        lista_fracassados.append(parecer_ajdg_homolog_tokenizado[i])
      if parecer_ajdg_homolog_tokenizado[i] == 'item' or parecer_ajdg_homolog_tokenizado[i] == 'itens' :
        capturar = False
        #cont = 0

cont = 0
for i in reversed(range(len(parecer_ajdg_homolog_tokenizado))):
    if parecer_ajdg_homolog_tokenizado[i].startswith('desert') and cont == 0:
      capturar = True
      cont = 1
    if capturar == True:
      if isinstance(parecer_ajdg_homolog_tokenizado[i], str) and parecer_ajdg_homolog_tokenizado[i].isdigit():
        lista_desertos.append(parecer_ajdg_homolog_tokenizado[i])
      if parecer_ajdg_homolog_tokenizado[i] == 'item' or parecer_ajdg_homolog_tokenizado[i] == 'itens' :
        capturar = False

lista_fracassados.reverse()
print(lista_fracassados)
lista_desertos.reverse()
print(lista_desertos)

['5', '12', '15', '22']
['2']


Final de captura dos números dos itens Desertos e Fracassados

listas: **lista_fracassados** e **lista_desertos**

Iniciar captura dos numeros dos itens do termo de referencia

In [ ]:
import re

# Definindo o padrão para capturar o trecho desejado
pattern = r'(ITEM.*?O objeto desta contratação não se enquadra como sendo de bem de luxo)'


# Encontrar o trecho entre os padrões
match = re.search(pattern, textPDF, re.DOTALL)

if match:
    # Obter o texto completo entre os padrões
    tabela_produtos = match.group(1).strip()  # O texto entre os dois padrões
else:
    print("Trecho não encontrado.")

In [ ]:
lista_textos_para_remover = [ "Câmara Nacional de Modelos de Licitações e Contratos da Consultoria-Geral da União",
                              "Aprovado pela Secretaria de Gestão e Inovação",
                              "UASG 70008"
                            ]

for texto in lista_textos_para_remover:
  tabela_produtos = tabela_produtos.replace(texto, "")

for i in range(1, num_pages + 1):
  tabela_produtos = tabela_produtos.replace(f"/ pg. {i}", "")

In [ ]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

tokens = word_tokenize(tabela_produtos)

# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
tabela_produtos_tokenizada = [word for word in tokens if word.lower() not in stop_words]
tabela_produtos_tokenizada = [item for item in tabela_produtos_tokenizada if 'UASG' not in item]

print(tabela_produtos_tokenizada)

['ITEM', 'DESCRIÇÃO', 'RESUMIDA', 'OBJETO', 'UNIDADE', 'QUANTIDADE', '1AAS', '–', 'Ácido', 'acetilsalicílico', '100', 'mg', ',', 'acondicionado', 'caixa', '20', 'comprimidos', '(', 'deverá', 'entregue', '1', 'caixa', '20', 'comprimidos', '2', 'envelopes', '10', 'unidades', 'cada', ')', '.Comprimido', '20', '2Adrenalina', '1mg', '–', 'epinefrina', '1', 'mg', 'Ampolas', '1', 'ml', '10', '3Allegra', 'D', '–', 'Cloridrato', 'Fexofenadina', '60', 'mg', '+', 'cloridrato', 'pseudoefedrina', '120', 'mg', '(', 'deverão', 'entregues', '10', 'caixas', '10', 'comprimidos', 'cada', ')', '.Comprimido', '100', '4Allegra', '180', 'mg', '–', 'Cloridrato', 'Fexofenadina', '180', 'mg', ',', 'acondicionado', 'caixa', '10', 'comprimidos', '(', 'deverão', 'entregues', '5', 'caixas', '10', 'comprimidos', 'cada', ')', '.Comprimido', '50', '5Andantol', 'gel', '–', 'Cloridrato', 'Isotipendil', '7.5', 'mg/g', 'Bisnaga', 'c/', '40g', '1', '6Bi-profenid', '150', 'mg', '–', 'Cetoprofeno', '150', 'mg', '(', 'deverão

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Obter o número do último produto da lista

filtered_list = [s for s in tabela_produtos_tokenizada if re.match(r'^[1-9][0-9A-Z.][a-zA-Z]', s)]
filtered_list = [s for s in filtered_list if not (len(s) <= 5 or 'UASG' in s)]
print(filtered_list)
ultimo = filtered_list[-1]
print(ultimo)
ultimo_produto = ''.join(filter(str.isdigit, ultimo))
ultimo_produto = int(ultimo_produto)
print(ultimo_produto)


['2Adrenalina', '3Allegra', '4Allegra', '5Andantol', '6Bi-profenid', '7Buscopan', '9Buscoduo', '10Buscopan', '11Captopril', '12Cefaliv', '14Dersani', '15Dersani', '16Dipirona', '17Dipirona', '18Domperidona', '19Dorflex', '20Dramin', '23Hixizine', '24Isordil', '25Luftal', '26Mirtax', '27Mylanta', '28Naramig', '29Nimesulida', '31Pantoprazol', '32Paracetamol', '33Pepsamar-', '34Plamet', '35Predsim', '37Reparil', '38Soro', '39Soro', '40Tiorfam', '41Tylenol', '42Vonau']
42Vonau
42


Criar lista com os números dos produtos homologados no processo licitatório

lista: **produtos_homologados**

In [ ]:
# Criar a nova lista com valores de 1 a ultimo_produto, excluindo os números da lista de exclusão
lista_desertos = [int(s) for s in lista_desertos]
lista_fracassados = [int(s) for s in lista_fracassados]

# Criar lista com os produtos que não foram homologados
lista_remover = lista_desertos + lista_fracassados

# Clista com os números dos produtos homologados
produtos_homologados = [i for i in range(1, int(ultimo_produto) + 1) if i not in lista_remover] # Changed ultimo_produto to int(ultimo_produto)
print(produtos_homologados)

[1, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]


Obtida a lista de produtos homologados: **produtos_homologados**

Iniciar a checagem se os produtos homologados estão em todas as notas de empenho

In [ ]:
#pegar números das notas de empenho
numeros_NE = []
for i in range (df_Notas_de_empenho.shape[0]):
  num_nota_de_empenho = df_Notas_de_empenho['NE'][i]
  integer_value = int(num_nota_de_empenho)
  numeros_NE.append(integer_value)
  numeros_NE = sorted(numeros_NE)

print(numeros_NE)

[322]


In [ ]:
#capturar as notas de empenho para a lista notas_de_empenho_completas

import re

notas_de_empenho_completas = []

for i in range (df_Notas_de_empenho.shape[0]):
  num_nota_de_empenho = numeros_NE[i]
  # Definindo o padrão para capturar o trecho desejado
  pattern = rf'(2024 NE {num_nota_de_empenho}.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO|2024 NE {num_nota_de_empenho}.*?ANA ESMERA PIMENTEL DA FONSECA)'
  #pattern = rf'(2024 NE.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO)'
  # pattern = rf'({num_nota_de_empenho}.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO)'

  # Encontrar o trecho entre os padrões
  match = re.search(pattern, textPDF, re.DOTALL)

  if match:
      # Obter o texto completo entre os padrões
      nota_empenho = match.group(1).strip()  # O texto entre os dois padrões
      notas_de_empenho_completas.append(nota_empenho)
  else:
      print(f"Trecho não encontrado para a NE {num_nota_de_empenho}")



In [ ]:
#inserir as notas de empenho tokenizadas e sem stopwords no DataFrame

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df_Notas_de_empenho['NE_Completa'] = ""

for i in range(len(notas_de_empenho_completas)):
  nltk.download('punkt')
  nltk.download('stopwords')

  # Supondo que 'text' é o conteúdo extraído do PDF
  tokens = word_tokenize(notas_de_empenho_completas[i])

  # Remover stopwords (palavras comuns que não têm significado relevante)
  stop_words = set(stopwords.words('portuguese'))
  df_Notas_de_empenho['NE_Completa'][i] = [word for word in tokens if word.lower() not in stop_words]

df_Notas_de_empenho

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-147-104fe0917522>:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas

,NE,CNPJ,Empresa,Valor,Email,NE_Completa
0,322,13.537.459/0001-70,2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA,"4.569,02",comercial@bigmedica.com.br,"[2024, NE, 322, Natureza, Despesa, Esfera, UGR..."


In [ ]:
#Capturar os números dos itens de cada nota de empenho e inserir no DataFrame

import numpy as np

df_Notas_de_empenho['itens_da_NE'] = ''

for i in range(df_Notas_de_empenho.shape[0]):
  num_NE = df_Notas_de_empenho['NE_Completa'][i][2]
  produtos = []
  for j in range(len(df_Notas_de_empenho['NE_Completa'][i])):
    if (df_Notas_de_empenho['NE_Completa'][i][j] == 'Item') and (df_Notas_de_empenho['NE_Completa'][i][j+1] == 'compra'):
      df_Notas_de_empenho.loc[df_Notas_de_empenho['NE'] == num_NE, 'itens_da_NE'] += df_Notas_de_empenho['NE_Completa'][i][j+3]
      df_Notas_de_empenho.loc[df_Notas_de_empenho['NE'] == num_NE, 'itens_da_NE'] += ', '
      # df_Notas_de_empenho['itens_da_NE'][i] += df_Notas_de_empenho['NE_Completa'][i][j+3]
      # df_Notas_de_empenho['itens_da_NE'][i] += ', '

# # Remove the last comma and space
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].str.rstrip(', ')

# # Split the string into a list of strings
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].str.split(', ')

# # Convert the list of strings to a list of integers
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].apply(lambda x: [int(i) for i in x])

df_Notas_de_empenho


,NE,CNPJ,Empresa,Valor,Email,NE_Completa,itens_da_NE
0,322,13.537.459/0001-70,2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA,"4.569,02",comercial@bigmedica.com.br,"[2024, NE, 322, Natureza, Despesa, Esfera, UGR...","[1, 3, 4, 6, 7, 8, 9, 9, 10, 11, 13, 14, 16, 1..."


In [ ]:
#remover repetidos

for i in range(df_Notas_de_empenho.shape[0]):
  df_Notas_de_empenho['itens_da_NE'][i] = list(set(df_Notas_de_empenho['itens_da_NE'][i]))

<ipython-input-149-dd9184e373c6>:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['itens_da_NE'][i] = list(set(df_Notas_de_empenho['itens_da_NE'][i]))


In [ ]:
#imprime a lista de produtos em cada Nota de Empenho

for i in range(df_Notas_de_empenho.shape[0]):
  print("----")
  print(f"Nota: {df_Notas_de_empenho['NE'][i]}")
  print(f"Produtos: {df_Notas_de_empenho['itens_da_NE'][i]}")

----
Nota: 322
Produtos: [1, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]


In [ ]:
#Criar uma lista com os produtos que foram homologados mas não estão em nenhuma Nota de Empenho

produtos_homologados_nao_presentes_nas_NE = produtos_homologados.copy()

for lista in df_Notas_de_empenho['itens_da_NE']:
  for num in lista:
    if num in produtos_homologados_nao_presentes_nas_NE:
      produtos_homologados_nao_presentes_nas_NE.remove(num)

#print(f'Produtos que foram homologados mas não estão em nenhuma NE: \n{produtos_homologados_nao_presentes_nas_NE}')

Criar o arquivo mensagem_itens_homologados.txt para listar os produtos que tenham sido homologados mas que não estão em nenhuma Nota de Empenho

In [ ]:
from datetime import datetime

# Obter a data atual
current_date = datetime.now()

# Formatar o dia, mês e ano como strings
formatted_date = f"SAMS - TRE-RN, {current_date.day}/{current_date.month}/{current_date.year}."

if len(produtos_homologados_nao_presentes_nas_NE) > 0:
  mensagem = f"Os produtos, {produtos_homologados_nao_presentes_nas_NE}, foram homologados mas não estão em nenhuma Nota de Empenho."
else:
  mensagem = f"Todos os produtos foram homologados e localizados em alguma Nota de Empenho."

# Nome do arquivo
nome_arquivo = "mensagem_itens_homologados.txt"

# Abrindo o arquivo em modo de escrita
with open(nome_arquivo, 'w') as arquivo:
    arquivo.write(mensagem)

#print(f"A mensagem foi escrita em {nome_arquivo}.")

#Início de criação da planilha de acompanhamento das entregas (Objetivo 3)

In [ ]:
#Cria df_para_planilha

from datetime import datetime, timedelta

# Obter a data atual
current_date = datetime.now()

# Adicionar 30 dias à data atual
new_date = current_date + timedelta(days=30)

formatted_date = new_date.strftime("%d/%m/%Y")

#resposta = input("Deseja criar a Planilha de Controle de Entregas? (s/n): ")
resposta = 's'

if resposta.lower() == 's':
  colunas_dropar = ['NE_Completa', 'itens_da_NE']
  df_para_planilha = df_Notas_de_empenho.drop(colunas_dropar, axis=1)
  df_para_planilha['Fim do prazo para entrega'] = formatted_date
  df_para_planilha['Data da Entrega'] = ''
  df_para_planilha['No. da Nota Fiscal'] = ''
  df_para_planilha['Pendências na Entrega'] = ''
  df_para_planilha['Enviado para pagamento em:'] = ''
  df_para_planilha['Pago em:'] = ''
  df_para_planilha.to_excel('Planilha de Controle de Entregas.xlsx', index=False)

In [ ]:
df_para_planilha

,NE,CNPJ,Empresa,Valor,Email,Fim do prazo para entrega,Data da Entrega,No. da Nota Fiscal,Pendências na Entrega,Enviado para pagamento em:,Pago em:
0,322,13.537.459/0001-70,2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA,"4.569,02",comercial@bigmedica.com.br,23/11/2024,,,,,


#Início da checagem de pagamentos (Objetivo 4)

In [ ]:
#fazendo split para pegar só a partir dos envios das notas de empenho

textPDF2 = textPDF.split('Envio de Nota de Empenho', 1)
if len(textPDF2) > 1:
  textPDF2 = textPDF2[1]
else:
  textPDF2 = ""


In [ ]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')


# Supondo que 'text' é o conteúdo extraído do PDF
tokens = word_tokenize(textPDF2)
# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
textPDF2_tokenizado = [word for word in tokens if word.lower() not in stop_words]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import re

lista_comprovantes = []

padrao1 = re.compile(r'^2024NE000\d{3}$')
padrao2 = re.compile(r'\d{2}[A-Z][a-z]{2}\d{2}')

for i in range (len(textPDF2_tokenizado)):
  if textPDF2_tokenizado[i] == '531814':
    comprovante = []
    for j in range (i+1, i+20):
      if padrao1.match(textPDF2_tokenizado[j]) or padrao2.match(textPDF2_tokenizado[j]):
        comprovante.append(textPDF2_tokenizado[j])
    lista_comprovantes.append(comprovante)

lista_comprovantes

lista_comprovantes_final = []
for sublist in lista_comprovantes:
  if sublist not in lista_comprovantes_final and len(sublist[1])<10:
    lista_comprovantes_final.append(sublist)


for lista in lista_comprovantes_final:
  lista[0] = lista[0][9:]

lista_comprovantes_final


[['322', '10Set24']]

In [ ]:
df_Notas_de_empenho = df_Notas_de_empenho.drop(columns=['NE_Completa'])

df_Notas_de_empenho['Data_Pag'] = 'Não'
df_Notas_de_empenho['NE'] = df_Notas_de_empenho['NE'].astype(str)

for i in range (len(lista_comprovantes_final)):
  for j in range (df_Notas_de_empenho.shape[0]):
    if lista_comprovantes_final[i][0] == df_Notas_de_empenho['NE'][j]:
      df_Notas_de_empenho['Data_Pag'][j] = lista_comprovantes_final[i][1]

df_Notas_de_empenho

<ipython-input-158-ea290ebfdcf2>:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['Data_Pag'][j] = lista_comprovantes_final[i][1]


,NE,CNPJ,Empresa,Valor,Email,itens_da_NE,Data_Pag
0,322,13.537.459/0001-70,2D XAVIER MEDICAMENTOS E PERFUMARIA LTDA,"4.569,02",comercial@bigmedica.com.br,"[1, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 16, 17, ...",10Set24


In [ ]:
df_Notas_de_empenho.to_excel('Planilha de Controle de Pagamentos.xlsx', index=False)

In [ ]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: blue;">Fim do Programa. Verifique a criação dos arquivos:</h1>'
text2 = '<h2 style="color: blue;">textos_emails.txt (Arquivo com os corpos dos emails para serem enviados).</h2>'
text3 = '<h2 style="color: blue;">mensagem_itens_homologados.txt (Arquivo com os itens que foram homologados mas não estão em nenhuma Nota de Empenho).</h2>'
text4 = '<h2 style="color: blue;">Planilha de Controle de Entregas.xlsx (Arquivo com a planilha de controle de entregas).</h2>'
text5 = '<h2 style="color: blue;">Planilha de Controle de Pagamentos.xlsx (Arquivo com a planilha de controle de pagamentos.</h2>'

# Exibir o texto
display(HTML(text1))
display(HTML(text2))
display(HTML(text3))
display(HTML(text4))
display(HTML(text5))